사생활을 보호하는 딥러닝에 필요한 기본 인프라를 만드는 `포인터 텐서`에 대해 배웠습니다. 이번 섹션에서는 이러한 기본 도구를 이용하여 `첫 사생활 보호 딥러닝 알고리즘`인 `연합 학습을 구현하는 방법`에 대해 살펴볼 것

### 연합 학습이란?

 연합 학습이란 딥러닝 모델을 학습하는 간단하고 강력한 방법입니다. 학습 데이터는 항상 일종의 수집 프로세스의 결과입니다. 사람들은 (각종 도구를 이용해) 실제 생활에서 이벤트를 기록하면서 데이터를 생성합니다. 일반적으로 이 데이터는 머신러닝 모델을 학습할 수 있도록 중앙의 한 곳으로 집계됩니다. 연합 학습은 이 방식을 완전히 바꿔놓습니다.

---

![image.png](https://blog.openmined.org/content/images/2019/02/Capture-d-e-cran-2019-02-25-a--17.45.36.png)

학습 데이터를 모델(중앙 서버)로 가져오는 대신에 **모델을 학습 데이터(사용 가능한 모든 곳)**로 가져가게 됩니다.

이를 통해 **데이터를 만드는 사람만이 영구 사본의 소유자가 되어서 액세스 권한이 있는 사람들을 제어**할 수 있게 됩니다. 근사하죠?


### Section 2.1 - 간단한 연합 학습 예제
--- 

연합학습을 다루기 전, 기존 중앙집중형식의 학습 방법을 다루보고 이와 비교해보자. 

먼저 **기존 학습 방법**에서 필요한 것은 다음과 같다.

- 간단한 데이터 셋
- 모델
- 데이터를 맞추기 위해 모델을 학습시키기 위한 몇 가지 기본 학습 방식

In [22]:
import torch
from torch import nn
from torch import optim

In [23]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [25]:
# A Toy Model
model = nn.Linear(2,1)

In [26]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [27]:
train()

tensor(0.3529)
tensor(0.2463)
tensor(0.1801)
tensor(0.1329)
tensor(0.0986)
tensor(0.0734)
tensor(0.0549)
tensor(0.0412)
tensor(0.0310)
tensor(0.0234)
tensor(0.0177)
tensor(0.0134)
tensor(0.0101)
tensor(0.0077)
tensor(0.0059)
tensor(0.0045)
tensor(0.0034)
tensor(0.0026)
tensor(0.0020)
tensor(0.0015)


완성입니다! 우리는 전통적인 방식으로 기본 모델을 학습시켰습니다. 모든 데이터는 로컬 컴퓨터로 집계되었으며 이를 사용하여 모델을 업데이트할 수 있습니다. 하지만 연합 학습은 이 방식으로 작동하지 않습니다. 

---

이 예제를 **연합 학습 방식으로 수정해 보자.**

그래서 우리가 필요한 것은 다음과 같다.

- 몇 개의 작업자 만들기
- 각 작업자의 학습 데이터에 대한 포인터를 얻기
- 연합 학습을 수행하기 위해 업데이트 된 학습 방법

    새로운 학습 단계:

    - 작업자에게 모델을 보냅니다.
    - 거기에 있는 데이터를 학습시킵니다.
    - 모델을 다시 가져오고 다음 작업자에서 이를 반복합니다.

In [55]:
import syft as sy
hook = sy.TorchHook(torch)

In [56]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [57]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [63]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            print('----------------------------------------------------------')
            print("data를 {0}에서 {1}로 보냄 ".format(data.owner, data.location))
            # NEW) send model to correct worker (# model 사전 정의 필요)
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [64]:
train()

----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <VirtualWorker id:bob #objects:6>로 보냄 
tensor(1.5504e-05, requires_grad=True)
----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <VirtualWorker id:alice #objects:6>로 보냄 
tensor(1.0409e-05, requires_grad=True)
----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <VirtualWorker id:bob #objects:6>로 보냄 
tensor(9.0866e-06, requires_grad=True)
----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <VirtualWorker id:alice #objects:6>로 보냄 
tensor(6.3690e-06, requires_grad=True)
----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <VirtualWorker id:bob #objects:6>로 보냄 
tensor(5.3755e-06, requires_grad=True)
----------------------------------------------------------
data를 <VirtualWorker id:me #objects:0>에서 <Virtu

### 학습 절차

1. 모델을 각 `worker`에게 보낸다.
2. 각 `worker`는 새로운 `gradient` 계산
3. 각 `worker`에서 새롭게 계산된 `gradient`를 `local server`로 가져와 `global model` 업데이트

### 위 예제의 단점

이 예제는 연합 학습에 대한 좋은 소개이지만 여전히 몇 가지 중요한 단점이 있습니다. 특히 model.get() 를 호출하고 Bob 또는 Alice로부터 업데이트 된 모델을 수신하면 실제로 우리는 그 그래디언트를 보면서 Bob과 Alice의 학습 데이터에 대해 많은 것을 배울 수 있습니다. 경우에 따라 학습 데이터를 완벽하게 복원도 가능합니다!

그래서 어떻게 해야할까요? 음, 사람들이 사용하는 첫번째 전략으로는 중앙 서버에 업로드하기 전에 여러 개인의 그래디언트를 평균화하는 것입니다. 그러나 이 전략을 사용하려면 PointerTensor 객체를 좀 더 정교하게 사용해야 합니다. 다음 섹션에서는 포인터의 고급 기능에 대해 배우고 연합 학습 예제를 업그레이드하도록 하겠습니다.